This program reads a tab separated file containing the author name affiliation, and generate the coordinates corresponding to their institude.

One line of the file looks like this:     
`Adam	Y.	Université de Liège	Liège	Belgium`      
The city column was added to make easier the localisation.

For each year, two output files are created:
1. a file storing only the coordinates, that can be used directly with leaflet (see [leaflet](./leaflet) folder) 
2. a file that is the same as the input file but with 2 extra columns for the coordinates.

In [1]:
import os
import glob
import logging
from collections import Counter
from liegecolloquium import Participant
from importlib import reload

# Configuration

In [2]:
datadir = "../data/"
outputdir = "../data/"
logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [3]:
filelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-1986.tsv")))
logging.info("Working on {0} files".format(len(filelist)))

In [4]:
participantlist = []
locationlist = []

for datafiles in filelist:
    logging.info("Working on file {0}".format(datafiles))
    outputbasename = os.path.basename(datafiles).replace(".tsv", ".dat")
    coordbasename = os.path.basename(datafiles).replace(".tsv", ".coord")
    outputfile = os.path.join(outputdir, outputbasename)
    coordfile = os.path.join(outputdir, coordbasename)
    logging.info("Writing info in file {0}".format(outputfile))
    logging.info("Writing coordinates in file {0}".format(outputfile))
    
    with open(coordfile, 'w') as cf:
        cf.write("var coords = [\n")

    with open(datafiles, 'r') as f:
        for line in f:
            
            if line == '\n':
                logger.debug("Empty line")
            else:
                l = line.rstrip().split('\t')
                participant = Participant(l[0], l[1], l[2], l[3], l[4])
                logger.info("Working with {0}".format(participant))

                # Modify country name via dictionary
                participant.replace_country()

                # Find location ; check if in the list before using geopy
                loc1 = ", ".join((participant.affiliation, 
                                  participant.city, 
                                  participant.country))
                logger.debug(loc1)

                if loc1 in locationlist:
                    logger.debug("Already in the list")
                    ind = locationlist.index(loc1)
                    logger.debug("Position in the list: {0}".format(ind))
                    participant.lat = participantlist[ind].lat
                    participant.lon = participantlist[ind].lon
                    participant.countryname = participantlist[ind].countryname
                    locationlist.append(locationlist.index(loc1))
                else:
                    logger.debug("Not in the list, use geolocator to get coordinates")
                    locationstring, location = participant.get_location()
                    locationlist.append(locationstring)

                # Write info to a text file
                participant.write_to(outputfile)

                # Write coordinates (only) into another file
                participant.write_coords_to(coordfile)

                # Append to list
                participantlist.append(participant)
            
    with open(coordfile, 'a') as cf:
        cf.write("]")

Management Unit of the Mathematical Model (MUMM), Liège, Belgium
INFO:locator:Management Unit of the Mathematical Model (MUMM), Liège, Belgium
Météorologie Nationale, Paris, France
INFO:locator:Météorologie Nationale, Paris, France
Universität Hamburg, Hamburg, Germany
INFO:locator:Universität Hamburg, Hamburg, Germany
Université de Laval, Québec, Canada
INFO:locator:Université de Laval, Québec, Canada
Universität Kiel, Kiel, Germany
INFO:locator:Universität Kiel, Kiel, Germany
Université de Liège, Liège, Belgium
INFO:locator:Université de Liège, Liège, Belgium
James Cook University of North Queensland, Townsville, Australia
INFO:locator:James Cook University of North Queensland, Townsville, Australia
Rosenstiel School of Marine and Atmospheric Sciences, Miami, United States of America
INFO:locator:Rosenstiel School of Marine and Atmospheric Sciences, Miami, United States of America
Institut de Protection et de Sûreté Nucléaire, CEA, Fontenay-aux-roses, France
INFO:locator:Institut de 

In [5]:
datafiles

'../data/ParticipantList-1986.tsv'

In [6]:
participant.affiliation

'Institute of Oceanographic Sciences'